In [1]:
import tqdm
# ^^^ pyforest auto-imports - don't write above this line
import numpy as np
import pandas as pd

from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, normalized_mutual_info_score
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.cluster import SpectralClustering

from sklearn.decomposition import PCA, SparsePCA, KernelPCA
from sklearn.manifold import TSNE

from rpy2.robjects import r, pandas2ri
from rpy2.robjects.vectors import StrVector

pandas2ri.activate()

import magic
import scprep

%matplotlib inline

# from sklearnex import patch_sklearn
# patch_sklearn()

import warnings

from sklearn.cluster import KMeans
from tqdm import tqdm

r('''
    source('../../imputation2/notebooks/repos/ALRA/alra.R')
''')

value,[RTYPES.CLOSXP]
visible,[RTYPES.LGLSXP]


In [2]:
def get_data(i):
    df = pd.read_csv('../data/{}/data.csv.gz'.format(i), index_col=0)
    tmp = np.sign(df)
    cols = (np.sum(tmp) > int((df.shape[0])*0.05))
    rows = (np.sum(tmp, axis=1) > int((df.shape[1])*0.05))
    df = np.log(df.loc[rows, cols] + 1)
    df_norm = df.copy()
    df_norm = scprep.normalize.library_size_normalize(df_norm)    
    df_norm = scprep.transform.sqrt(df_norm)
    X_norm = pd.DataFrame(df_norm, columns=df.columns)
    labels = df.index
    return X_norm, labels

In [3]:
X_norm, labels = get_data('brosens')
r.assign("A", X_norm)
r('''
    A = as.matrix(A)
''')
r('''
    # Library and log normalize the data
    A_norm <- normalize_data(A)
    # Choose k. 
    k_choice <- choose_k(A_norm)
''')
pred = r('''
    A_norm_completed <- alra(A_norm,k=k_choice$k)[[3]]
''')

Read matrix with 3567 cells and 8102 genes
Getting nonzeros
Randomized SVD
Find the 0.001000 quantile of each gene
Sweep
Scaling all except for 0 columns
0.00% of the values became negative in the scaling process and were set to zero
The matrix went from 26.43% nonzero to 81.40% nonzero


In [5]:
pd.DataFrame(pred, ).to_csv('result/alra.csv')